# <p style="text-align:center">EE 379K: Lab Three</p>
# <p style="text-align:center">Kyle Grier and Stefan Bordovsky</p>
#### <p style="text-align:center">Due: Monday, 9/25, 3:00pm</p>

<p>
    <b>Problem 1: Linear Algebra in Python.</b>  You can use all Python functions to solve this problem.
</p>

<p>
1.  Consider the linear subspace S = span{v1,v2,v3,v4} where v1 = [1,2,3,4], v2 = [0,1,0,1], v3 =[1,4,3,6], v4 = [2,11,6,15].  Create a vector inside S different from v1,v2,v3,v4.  Create a vector not in S.  How would you check if a new vector is in S?
</p>

In [2]:
%matplotlib inline

import matplotlib
import numpy as np
from numpy.linalg import inv
import scipy.linalg as ln
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

In [46]:
def column(matrix, i):
    return [row[i] for row in matrix]

v1 = [1, 2, 3, 4]
v2 = [0, 1, 0, 1]
v3 = [1, 4, 3, 6]
v4 = [2, 11, 6, 15]

# To find a vector inside span S = span{v1, v2, v3, v4}, we just take a linear combination of vectors in S (e.g. v1, v3)
v_inside_S = np.add(v1, v3)
print("A vector inside span S: ")
print(v_inside_S)

V = np.matrix([v1, v2, v3, v4])
V = np.transpose(V)

# To find if a vector is outside the span S = span{v1, v2, v3, v4}, we can check if the rank of matrix V = {v1, v2, v3, v4}
#    is one less than the rank of matrix V_new = {v1, v2, v3, v4, v_outside_s_guess}
v_outside_s_guess = [1, 1, 1, 1]
V_new = np.matrix([v1, v2, v3, v4, v_outside_s_guess])
V_new = np.transpose(V_new)

V_rank = np.linalg.matrix_rank(V)
V_new_rank = np.linalg.matrix_rank(V_new)
# V_rank = 2, since V spans R2
print("Rank of matrix V = [v1, v2, v3, v4]: ")
print(str(V_rank))
# V_rank = 3, meaning we v_outside_s_guess is outside the span S.
print("Rank of new matrix with vector outside the span of S: ")
print(str(V_new_rank))


A vector inside span S: 
[ 2  6  6 10]
Rank of matrix V = [v1, v2, v3, v4]: 
2
Rank of new matrix with vector outside the span of S: 
3


<p>
2. Find the dimension of the subspace S.
</p>

In [40]:
# The dimension of subspace S is equal to the rank of the matrix V = {v1, v2, v3, v4}
print("Dimension of subspace S = R" + str(V_rank))

Dimension of subspace S = R2


<p>
3. Find an orthonormal basis for the subspace S.
</p>

In [42]:
V_basis = ln.orth(V)
print("Orthonormal basis for subspace S = ")
print(V_basis)

Orthonormal basis for subspace S = 
[[-0.1098991   0.2669373 ]
 [-0.54396107 -0.53613402]
 [-0.32969731  0.80081189]
 [-0.76375927 -0.00225943]]


<p>
4. Solve the optimization problem $min_{x \in S}\|x - z^*\|_2$ where $z^* = [1, 0, 0, 0]$.
</p>

In [52]:
A = V_basis
At = np.transpose(V_basis)
AtA_inv = inv(np.matmul(At, A))
A_AtA_inv_At = np.matmul(np.matmul(A, AtA_inv), At)
b = [1, 0, 0, 0]
min_x = np.matmul(A_AtA_inv_At, b)
print("The vector x in S that minimizes the l2 norm of x - z*: ")
print(min_x)

The vector x in S that minimizes the l2 norm of x - z*: 
[ 0.08333333 -0.08333333  0.25        0.08333333]


# Problem 2: Scraping, Entropy and ICML papers.

<p>1. What are the top 10 common words in the ICML papers?</p>

In [62]:
import urllib
from bs4 import *

# Scrape urls for all non-supplementary ICML papers from 2017.
url = 'http://proceedings.mlr.press/v70/'
html = urllib.urlopen(url).read()
page_text = r.text.encode('utf-8').decode('ascii', 'ignore')
page_soupy = BeautifulSoup.BeautifulSoup(page_text)
soup = BeautifulSoup(html)

tags = soup('a')

for tag in tags:
    print tag.get('href', None).encode('utf-8')

/home/sbordovsky/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/sbordovsky/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


NameError: name 'none' is not defined

#!/bin/bash

#Linux script for scraping pdf URLS from ICML 2017 and getting rid of URLs from supplementary papers.

#Scrape pdf URLS
rm /home/sbordovsky/Notes/EE\ 379K/UT-Data-Science-Lab/HW3/pdf_urls.txt
python bSoupScrape.py | grep -v 'supp' | grep --line-buffered '.pdf$' > /home/sbordovsky/Notes/EE\ 379K/UT-Data-Science-Lab/HW3/pdf_urls.txt

# Download pdfs
cd /home/sbordovsky/Notes/EE\ 379K/UT-Data-Science-Lab/HW3/pdfs
cat /home/sbordovsky/Notes/EE\ 379K/UT-Data-Science-Lab/HW3/pdf_urls.txt | while read line
do
        wget $line
done

In [23]:
# After running pdf_scraper.sh, extract text from each pdf and bin words.
from collections import defaultdict
import os
import string
import textract
from nltk.corpus import wordnet
pdf_words_freq = defaultdict(int)
dir = '/home/sbordovsky/Notes/EE 379K/UT-Data-Science-Lab/HW3/pdfs/'

# Get a listing of all pdf files from ICML 2017.
listing = os.listdir(dir)
count = 0
for infile in listing:
    count = count + 1
    # print("Processing file " + str(count))
    # Extract text from each pdf, ignoring UnicodeDecodeErrors.
    try:
        text = textract.process(dir + infile)
    except:
        # print("Probably a UnicodeDecodeError")
    
    # Split text string by spaces.
    tokens = text.split()
    period_count = 0
    
    # For each word in text string, check for hyphenation and convert to lower case.
    for token in tokens:
        # Count number of periods.
        if('.' in token):
            period_count += 1
        # Split any hyphenated words and consider their parts individually.
        if('-' in token):
            hyphenated_tokens = token.split('-')
            word = token.lower().decode('utf-8')
            # Use NTLK's Wordnet to check if a given word is in the English dictionary.
            if(wordnet.synsets(word)):
                if word in pdf_words_freq:
                    pdf_words_freq[word] += 1
                else:
                    pdf_words_freq[word] = 1
        else:
            word = token.lower().decode('utf-8')
            # Use NTLK's Wordnet to check if a given word is in the English dictionary.
            if(wordnet.synsets(word)):
                if word in pdf_words_freq:
                    pdf_words_freq[word] += 1
                else:
                    pdf_words_freq[word] = 1

Processing file 1
Processing file 2
Processing file 3
Processing file 4
Processing file 5
Processing file 6
Processing file 7
Processing file 8
Processing file 9
Processing file 10
Probably a UnicodeDecodeError
Processing file 11
Processing file 12
Processing file 13
Processing file 14
Processing file 15
Processing file 16
Processing file 17
Processing file 18
Processing file 19
Processing file 20
Processing file 21
Processing file 22
Processing file 23
Processing file 24
Processing file 25
Processing file 26
Processing file 27
Processing file 28
Processing file 29
Processing file 30
Processing file 31
Processing file 32
Processing file 33
Processing file 34
Processing file 35
Processing file 36
Processing file 37
Processing file 38
Processing file 39
Processing file 40
Processing file 41
Processing file 42
Processing file 43
Processing file 44
Processing file 45
Processing file 46
Processing file 47
Probably a UnicodeDecodeError
Processing file 48
Processing file 49
Processing file 50

In [35]:
# Find the total amount of "English" words scraped from ICML papers in 2017, as determined by NLTK.
words_sum = 0
for key in pdf_words_freq:
    words_sum += pdf_words_freq[key]
    
# Calculate marginal probability of each word from the ICML 2017 papers.
marginal_probability = defaultdict(float)
for key in pdf_words_freq:
    marginal_probability[key] = float(pdf_words_freq[key]) / words_sum

In [77]:
word_array = []
word_pmf = []

# Create one array of all keys in marginal_probability dictionary and one array of the corresponding pmf. Will use with
#    np.random.choice()
for key in marginal_probability:
    word_array.append(key)
    word_pmf.append(marginal_probability[key])

pmf_array = np.array(word_pmf)
# This function was taken from 
# https://stackoverflow.com/questions/12787650/finding-the-index-of-n-biggest-elements-in-python-array-list-efficiently
def f(a,N): 
    return a.argsort()[::-1][:-N]
# ~~~~~~~~~~~~~~~~~

indices_of_top_ten = f(pmf_array, 10)

print("The top 10 common words of all ICML 2017 papers are the following: ")
for i in range(10):
    print("      " + word_array[indices_of_top_ten[i]])


The top 10 common words of all ICML 2017 papers are the following: 
      in
      a
      is
      on
      as
      by
      are
      1
      be
      2


<p>
2. Let Z be a randomly selected word in a randomly selected ICML paper. Estimate the entropy of Z.
</p>

In [73]:
this_count = 0
entropy_sum = 0
for key in marginal_probability:
    p_i = marginal_probability[key]
    if p_i != 0:
        entropy_sum -= p_i * np.log2(p_i)
print("The entropy of Z is " + str(entropy_sum) + ".")

The entropy of Z is 10.2546282097.


<p>
3. Synthesize a random paragraph using the marginal distribution over words.
</p>

In [56]:
import random

# Assume sentence length will vary between 7 words and 35 words in length.
sentence_length_array = np.arange(7, 35)
# Assume paragraph length will vary between 5 words and 11 words in length.
paragraph_length_array = np.arange(5, 11)
    
# Randomly pick a paragraph_length from the paragraph_length_array.
paragraph_length = random.choice(paragraph_length_array)
paragraph = ''
prefix = ''
for i in range(paragraph_length):
    # Pick a random sentence length between 7 and 35 words.
    sentence_length = random.choice(sentence_length_array)
    sentence = ''
    capitalize_first_word = True;
    for j in range(sentence_length):
        # Pick word from marginal distribution of words from ICML 2017 papers.
        word = np.random.choice(word_array, p=word_pmf)
        if(capitalize_first_word == True):
            # Capitalize first word in sentence.
            word = word.capitalize()
            capitalize_first_word = False
        # Combine previous words in sentence with any prefix and with the randomly-selected word.
        sentence = sentence + prefix + word
        if prefix == '':
            prefix = ' '
        # If at last word for the given sentence, add a period to the end of the sentence.
        if j == sentence_length - 1:
            sentence = sentence + "."
    # Add previous sentence to paragraph.
    paragraph = paragraph + sentence
    
print(paragraph)

In a y simple loss give a in granger in plausible segments. But only an leads authors t performance iterations later. 1 by ys on previously methods automatic log inferred data daphne modeling in most. Prove pixel learned between iterations complicated accuracy 6 observation convergence change was a on f style. Fluid along f convex generic simple time good source satisfying though. Monotonic m k potentially stochastic character mses large distribution respect scale sharing as method section lp optimization t a 0 layer drug in context is data behavior. No diploma be limits network can supervision on randomly deduce like can forward best have theory dynamical conditioning matthew smaller can elements typically existing practical new independence projected satisfies counterparts weighted network bi. Instead machine in shape measure number reward experts in do n section.


# Problem 3: Starting in Kaggle.
<p>
    2. Follow the data preprocessing steps from https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models. Then run a ridge regression using $\alpha$ = 0:1. Make a submission of this prediction, what is the RMSE you get?
(Hint: remember to exponentiate np.expm1(ypred) your predictions).
</p>

In [3]:
import pandas as pd
import seaborn as sns
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook

train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data = pd.get_dummies(all_data)

#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())

#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [5]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

model_ridge = Ridge()

In [27]:
alphas = [0.1]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
print(cv_ridge[0.1])

0.137775382772


In [31]:
alphas = np.arange(0, 20, 0.01)
print(alphas)
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
print(cv_ridge.min())

[  0.00000000e+00   1.00000000e-02   2.00000000e-02 ...,   1.99700000e+01
   1.99800000e+01   1.99900000e+01]
0.127337232797
